In [1]:
!pip install schedule

In [ ]:
from selenium import webdriver
import schedule
import time
import sqlite3
from bs4 import BeautifulSoup


def job():

    driver = webdriver.Chrome('./chromedriver.exe')
    conn = sqlite3.connect('./db.stock')
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , href TEXT, stock TEXT, posi_nega TEXT)")

    uri = 'https://finance.naver.com/news/market_special.nhn?&page='
    fnews = []

    for page in range(1, 3):
        target = uri+str(page)
        driver.get(target)

        try:
            for n in range(1, 24):
    #             time.sleep(2)
                driver.implicitly_wait(5)
                try: 
                    driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/table/tbody/tr[%s]/td[1]/a' %n).click()
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    presses = soup.select('span.press > img')
                    press = presses[0]['title']

                    datas = soup.select('div.article_info > h3')
                    title = datas[0].text.strip()

                    dates = soup.select('div.article_sponsor > span')
                    date = dates[0].text.strip()

                    contents = soup.select('div#content')
                    content1 = contents[0]
                    for tag in content1.find_all(['li', 'h3']):
                        tag.replace_with('')
                        content = content1.text.strip()

                    driver.back()

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    times = soup.select('table > tbody > tr:nth-child(%s) > td.wdate' %n)
                    time = times[0].text.strip()[-5:]
                    hrefs = soup.select('tr:nth-child(%s) > td.publicSubject > a' %n)
                    href = 'https://finance.naver.com'+ hrefs[0]['href']
                    
                    a = [date , time , press , title, content, href]
                    fnews.append(a)

                    c.execute("INSERT INTO article( date , time , title, content, press, href, stock , posi_nega ) VALUES(?,?,?,?,?,?,?,?)",(date, time, title, content, press, href,'stock','posi_nega'))

                except:
                    pass

        except:
            pass  

    print(fnews)
    conn.commit()
    c.close()

    
schedule.every().day.at("17:41").do(job) 

while True:
    schedule.run_pending()
    time.sleep(5)


[['2021-08-17', '15:56', '연합뉴스', '한컴라이프케어, 공모가 대비 -6.6%…딥노이드 하한가(종합)', '(서울=연합뉴스) 박진형 기자 = 한컴라이프케어·딥노이드가 상장 첫날인 17일 각각 급락 마감했다.    이날 유가증권시장에서 한컴라이프케어는 시초가(1만6천50원)보다 20.25% 떨어진 1만2천800원에 거래를 마쳤다.한컴라이프케어[IFG파트너스 제공]    공모가(1만3천700원)보다도 6.57% 내렸다.    코스닥시장에서 딥노이드도 거래제한폭(-30.00%)까지 떨어진 5만400원에 마감했다.    다만 공모가(4만2천원)와 비교하면 20.00% 높다.    한컴라이프케어는 공기호흡기, 방독면, 방역복, 산업·보건 마스크 등을 제조하는 안전장비 전문업체다.    딥노이드는 의료 인공지능(AI) 솔루션 기업이다.딥노이드[IR큐더스 제공. 재판매 및 DB금지]    jhpark@yna.co.kr', 'https://finance.naver.com/news/market_special_read.nhn?article_id=0012604095&office_id=001&type=title&keyword='], ['2021-08-17', '15:51', '연합뉴스', '삼성바이오 첫 100만원 돌파…SK바이오사이언스 신고가(종합)', '삼성바이오로직스 외관삼성바이오로직스 외관. 2018.05.24. [삼성바이오로직스 제공=연합뉴스](서울=연합뉴스) 김태종 기자 = 삼성바이오로직스와 SK바이오사이언스가 모더나 백신 위탁 생산과 아스트라제네카(AZ) 백신 출하 소식에 17일 신고가를 기록했다.     이날 유가증권시장에서 삼성바이오로직스는 전날보다 2.95%(2만9천원) 상승한 101만2천원에 거래를 마감했다.    삼성바이오로직스가 100만원을 넘은 것은 이번이 처음으로, 장중에는 102만원까지 치솟기도 했다.    이날 외국인과 기관이 각각 3만6천주와 1만7천주를 순매수했다.    시가총액도 66조9천590억원으로 불어나며, 순위에서도 